In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=d03b59395f03bfeb5d59ba80c0ef911dd6cf83e117bbff11e790d20335e7708d
  Stored in directory: /root/.cache/pip/wheels/1e/5c/47/759b32beb377aee150eb4c1f1b2b7635032cc19ea09ce076fd
Successfully built pandas_ta


In [ ]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [ ]:
import plotly.graph_objs as go
#import talib as ta
import pandas_ta as ta
import datetime as dt
import pandas as pd
import numpy as np

name = '積水ハウス'
code = 1928
df = get_stock_data(code)
close = df["Close"]

# 5日、25日移動平均の算出
#ma5, ma25 = ta.SMA(close, timeperiod=5), ta.SMA(close, timeperiod=25)
ma5, ma25 = ta.sma(close ,length=5), ta.sma(close ,length=25)
df["ma5"], df["ma25"] = ma5, ma25

ma5, ma25 = df["ma5"], df["ma25"]
cross  = ma5 > ma25

cross_shift = cross.shift(1)
temp_gc = (cross != cross_shift) & (cross == True)
temp_dc  = (cross != cross_shift) & (cross == False)

# ゴールデンクロス発生日であればMA5の値、それ以外はNaN
gc = [m if g == True else np.nan for g, m in zip(temp_gc, ma5)]

# デッドクロス発生日であればMA25の値、それ以外はNan
dc = [m if d == True else np.nan for d, m in zip(temp_dc, ma25)]

# データフレームのカラムとして保存
df["gc"], df["dc"] = gc, dc

# ボリンジャーバンドの算出
upper2, _, lower2 = ta.BBANDS(close, timeperiod=25, nbdevup=2, nbdevdn=2, matype=0)
df['upper2'], df['lower2'] = upper2, lower2

bb2 = ta.bbands(close, length=25, std=2.0, mamode="sma")
df["upper2"], df["lower2"] = bb2["BBU_25_2.0"], bb2["BBL_25_2.0"]

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

layout = {
            "height": 700,
            "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
            "xaxis" : { "rangeslider": { "visible": False } },
            "yaxis1": { "domain": [.20, 1.0], "title": "価格（円）", "side": "left", "tickformat": "," },
            "yaxis2": { "domain": [.10, .20] },
            "yaxis3": { "domain": [.00, .10], "title": "Volume", "side": "right" },
            "plot_bgcolor":"light blue"
          }

data =  [
            # ローソク足
            go.Candlestick(yaxis="y1", x=rdf.index, open=rdf["Open"], high=rdf["High"], low=rdf["Low"], close=rdf["Close"],   
                            increasing_line_color="red", decreasing_line_color="gray"),
            # 5日移動平均線
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma5"], name="MA5", line={ "color": "royalblue", "width": 1.2 } ),
            # 25日移動平均線
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma25"], name="MA25", line={ "color": "lightseagreen", "width": 1.2 } ),
            # ゴールデンクロス
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["gc"], name="Golden Cross", mode="markers", marker={ "size": 15, "color": "purple" }, opacity = 0.5),
            # デッドクロス
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["dc"], name="Dead Cross", mode="markers", marker={ "size": 15, "color": "black", "symbol": "x" },  opacity = 0.8),         
            # ボリンジャーバンド
            go.Scatter(yaxis="y1", x=rdf.index ,y=rdf["upper2"], name= "", line={ "color": "lavender", "width": 0 }),
            go.Scatter(yaxis="y1", x=rdf.index ,y=rdf["lower2"], name= "BB", line={ "color": "lavender", "width": 0}, 
                    fill="tonexty", fillcolor="rgba(170,170,170,.2)"),
            # 出来高
            go.Bar(yaxis="y3", x=rdf.index, y=rdf["Volume"], name= "Volume", marker={ "color": "slategray" } ) 
        ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

AttributeError: ignored